In [1]:
%load_ext autoreload
%autoreload 2

from Shared.shared import *
from Shared.specific_CNB_sim import *

import pandas as pd

sim_name = f"Dopri5_1k_z1"
sim_folder = f"sim_output/{sim_name}"
fig_folder = f"figures_local/{sim_name}"
Cl_folder = f"Shared/Cls"
nu_m_range = jnp.load(f"{sim_folder}/neutrino_massrange_eV.npy")
nu_m_picks = jnp.array([0.01, 0.05, 0.1, 0.2, 0.3])*Params.eV
simdata = SimData(sim_folder)

z_int_steps = jnp.load(f"{sim_folder}/z_int_steps.npy")
s_int_steps = jnp.load(f"{sim_folder}/s_int_steps.npy")
print(z_int_steps)

# nu_vecs_FD = jnp.load(f"{sim_folder}/vectors_day1.npy")
# pix_dens_FD = jnp.load(f"{sim_folder}/pixel_densities_day1.npy")
# tot_dens_FD = jnp.load(f"{sim_folder}/total_densities_day1.npy")
# print(nu_vecs_FD[0,0,...])
# print(pix_dens_FD.shape)
# print(tot_dens_FD)

[0.         0.00252524 0.00511425 0.00776864 0.01049006 0.01328021
 0.01614081 0.01907365 0.02208054 0.02516337 0.02832405 0.03156455
 0.03488687 0.03829309 0.04178533 0.04536575 0.04903659 0.05280013
 0.0566587  0.06061472 0.06467063 0.06882896 0.0730923  0.0774633
 0.08194468 0.08653923 0.0912498  0.09607932 0.1010308  0.10610731
 0.11131202 0.11664817 0.12211906 0.12772811 0.13347879 0.1393747
 0.14541949 0.15161693 0.15797087 0.16448526 0.17116416 0.17801171
 0.18503218 0.19222994 0.19960945 0.20717532 0.21493224 0.22288505
 0.23103868 0.23939821 0.24796884 0.2567559  0.26576485 0.2750013
 0.28447099 0.29417982 0.30413382 0.31433918 0.32480225 0.33552954
 0.34652772 0.35780363 0.36936428 0.38121687 0.39336876 0.40582752
 0.4186009  0.43169683 0.44512347 0.45888916 0.47300247 0.48747217
 0.50230727 0.51751699 0.5331108  0.54909838 0.5654897  0.58229493
 0.59952453 0.61718923 0.6353     0.65386811 0.67290511 0.69242284
 0.71243344 0.73294936 0.75398336 0.77554851 0.79765824 0.8203262

In [10]:
t_Universe = 2*Params.t0/Params.s
G_Universe = 1/t_Universe
print(G_Universe)
56/2

1.149988829468506e-18


28.0

[[56.59817452 56.60432873 56.60716104 56.60837604 56.60885944 56.60903686
  56.60909659 56.60911493 56.60912002 56.6091213  56.60912158 56.60912164
  56.60912165 56.60912165 56.60912165 56.60912165 56.60912165 56.60912164
  56.60912164 56.60912164 56.60912164 56.60912164 56.60912163 56.60912163
  56.60912162 56.60912162 56.60912161 56.6091216  56.60912159 56.60912157
  56.60912155 56.60912153 56.6091215  56.60912146 56.60912142 56.60912137
  56.60912131 56.60912123 56.60912113 56.60912101 56.60912086 56.60912068
  56.60912045 56.60912012 56.60911983 56.60911941 56.60911889 56.60911825
  56.60911746 56.60911649]]

### CNB in solar region.

In [ ]:
ES_dist_av = 150.27e6*Params.km
nu_vel_max = 0.05*Params.c
nu_ES_time = ES_dist_av/nu_vel_max
time_unit = Params.s*60*60*24
print(nu_ES_time/time_unit)

### Neutrino mass hierarchies.

In [ ]:
### ====================================== ###
### Global loop over neutrino mass choices ###
### ====================================== ###

# We later always select the first of the 3 masses, to generate the Cl's.
# In that way we get Cl's precisely for first mass. Class still needs 3 masses.

nu_mass_set0 = Physics.neutrino_masses(
    m_lightest=0.00001, ordering='NO', args=Params())

# To get 0.01 eV
nu_mass_set1 = Physics.neutrino_masses(
    m_lightest=0.01, ordering='NO', args=Params())

# To get 0.05 eV
nu_mass_set2 = Physics.neutrino_masses(
    m_lightest=0.05, ordering='NO', args=Params())

# To get 0.1 eV
nu_mass_set3 = Physics.neutrino_masses(
    m_lightest=0.1, ordering='NO', args=Params())

# To get 0.2 eV
nu_mass_set4 = Physics.neutrino_masses(
    m_lightest=0.2, ordering='NO', args=Params())

# To get 0.3 eV
nu_mass_set5 = Physics.neutrino_masses(
    m_lightest=0.3, ordering='NO', args=Params())

nu_masses_sets = [
    nu_mass_set0.tolist(),
    nu_mass_set1.tolist(),
    nu_mass_set2.tolist(),
    nu_mass_set3.tolist(),
    nu_mass_set4.tolist(),
    nu_mass_set5.tolist(),
]

nu_masses = nu_masses_sets[0]
print(nu_masses)

# Always selecting first mass out of the 3
m_idx = 0
m_val = nu_masses[m_idx]

print(fr"*** Generating Cl for $m_\nu$={m_val} eV ***")

In [ ]:
@chex.dataclass
class Test:
    # Temperature of CMB
    T_CMB: float = 2.72548

    # N_eff parameter, using results of https://doi.org/10.1103/PhysRevD.108.L121301
    N_eff: float = 3.043

    # Temperature of CNB
    T_CNB = jnp.power(4/11, 1/3)*T_CMB * jnp.power(N_eff/3, 1/4)

    # Temperature of CNB
    def T_CNB_fct(N_eff=N_eff, T_CMB=T_CMB):
        return jnp.power(4/11, 1/3)*T_CMB * jnp.power(N_eff/3, 1/4)


print(Test.T_CNB_fct())
print(Test.T_CNB_fct(N_eff=3.0))

print("END")